In [11]:
import time
import dask
from dask.distributed import Client
import os
import subprocess
from pathlib import Path
import pprint
from typing import List

In [2]:
%%bash
python --version 

Python 3.8.10


In [3]:
pp = pprint.PrettyPrinter()

In [12]:
def task(cmd: List[str]):
    # log file name
    stdout = Path.cwd() / "stdout.txt"
    out_stream = open(stdout, 'wb')

    # error file name
    stderr = Path.cwd() / "stderr.txt"
    err_stream = open(stderr, 'wb')

    pid = os.getpid()
    # cmd = ["ls", "-l"]
    out = subprocess.run(cmd, shell=True, capture_output=True, text=True)
    return out

In [5]:
dask.config.config["distributed"]["dashboard"]["link"] = "{JUPYTERHUB_SERVICE_PREFIX}proxy/{host}:{port}/status" 

scheduler_file = os.path.join(os.environ["SCRATCH"], "scheduler_file.json")
# create a client
client = Client(scheduler_file=scheduler_file)

/usr/local/lib/python3.8/dist-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [6]:
client

<Client: 'tcp://10.249.44.177:8786' processes=128 threads=32768, memory=59.57 TiB>

## Testing Job

In [7]:
# submit a testing task
num_tasks = 10_000
futures = []
for i in range(num_tasks):
    futures.append(client.submit(task, ["ls", "-l"]))

# wait for the task to finish
futures = client.gather(futures)

In [8]:
print(len(futures))
pp.pprint(futures[1].stdout)

10000
('Dask_Example-Container.ipynb\n'
 'Dask_Example.ipynb\n'
 '__pycache__\n'
 'create_ray_jobs.py\n'
 'dask_client.py\n'
 'detectors_odd.csv\n'
 'full_chain_odd.py\n'
 'get_py_cmd.sh\n'
 'jobs.txt\n'
 'launch-dask-shifter-cpu.sh\n'
 'launch-dask-shifter-gpu.sh\n'
 'odd-sp-config.json\n'
 'prepare.py\n'
 'run_acts_in_shifter.sh\n'
 'run_generic.sh\n'
 'stderr.txt\n'
 'stdout.txt\n'
 'submit.sh\n')


## ACTS jobs

In [28]:
num_jobs = 400
num_evts = 10
outdir = "dask_v5"

# cmd = ["./run_acts_in_shifter.sh", str(num_evts), outdir]
cmd = f"./run_acts_in_shifter.sh {num_evts} {outdir}"

In [29]:
futures = []
for i in range(num_jobs):
    seed = 756323 + i
    full_cmd = cmd + f" {seed}"
    # pp.pprint(full_cmd)
    futures.append(client.submit(task, full_cmd))

# wait for the task to finish
futures = client.gather(futures)

KeyboardInterrupt: 

In [30]:
futures[0]

<Future: pending, key: task-ed6ba7cc9bde21a48637ec89e88edfdc>

Task exception was never retrieved
future: <Task finished name='Task-12614' coro=<Client._gather.<locals>.wait() done, defined at /usr/local/lib/python3.8/dist-packages/distributed/client.py:2189> exception=AllExit()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/distributed/client.py", line 2198, in wait
    raise AllExit()
distributed.client.AllExit
Task exception was never retrieved
future: <Task finished name='Task-12625' coro=<Client._gather.<locals>.wait() done, defined at /usr/local/lib/python3.8/dist-packages/distributed/client.py:2189> exception=AllExit()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/distributed/client.py", line 2198, in wait
    raise AllExit()
distributed.client.AllExit
Task exception was never retrieved
future: <Task finished name='Task-12734' coro=<Client._gather.<locals>.wait() done, defined at /usr/local/lib/python3.8/dist-packages/distributed/client.py:2189> exception=AllExit()>
Tr